In [2]:
import numpy as np
import chess 
import chess.pgn
import random
from datetime import datetime
import pickle

In [2]:
def set_seed(seed_num):
    '''
    Sets the seed for the python random module
    '''
    random.seed(seed_num)
    
def outcome(game):
    '''
    Returns the results of a game.
        - 0: White won
        - 1: Black won
        - 2: Draw
    '''
    return str(game.headers['Result'][-1])

def sample_position(position_list, move_list, num_positions, num_omit):
    '''
    Returns a list of candidate positions for training our model. These positions must
    fit the criteria of:
        - Only taken after first num_omit steps,
        - Does not involve a capture move
    '''
    # Only look for moves after first num_omit steps
    pos = position_list[num_omit:]
    moves = move_list[num_omit:]
    
    # Exclude moves that involved a capture (i.e. has an 'x' notation in the moves list)
    pos = [pos[i] for i in range(len(moves)) if 'x' not in moves[i]]
    
    if len(pos) > num_positions:
        return random.sample(pos, num_positions)
    
    return pos
    
def pgn_to_fen(game):
    '''
    Reads a game and parses through the moves of the entire game. Returns two 
    lists, position_list and move_list which contains a list of FEN and SAN notation respectively.
    '''
    node = game
    position_list = []
    move_list = []
    
    while not node.is_end():
        next_node = node.variation(0)
        position_list.append(next_node.board().fen())
        move_list.append(node.board().san(next_node.move))
        node = next_node
        
    return position_list, move_list

def fen_to_bitboard(fen):
    '''
    Takes in a FEN position and returns a bitboard notation
    '''
    
    chess_pieces = {
        'p': 0,
        'n': 1,
        'b': 2,
        'r': 3,
        'q': 4,
        'k': 5,
        'P': 6,
        'N': 7,
        'B': 8,
        'R': 9,
        'Q': 10,
        'K': 11
    }
    
    bitboard = [0]*773
    currIndex = 0
    [position, turn, castling, _, _, _] = fen.split(' ')
    for ch in position:
        if ch == '/':
            continue
        elif ch >= '1' and ch <= '8':
            currIndex += (ord(ch) - ord('0')) * 12
        else:
            bitboard[currIndex + chess_pieces[ch]] = 1
            currIndex += 12
    bitboard[768] = 1 if turn == 'w' else 0
    bitboard[769] = 1 if 'K' in castling else 0
    bitboard[770] = 1 if 'Q' in castling else 0
    bitboard[771] = 1 if 'k' in castling else 0
    bitboard[772] = 1 if 'q' in castling else 0
    return bitboard

def games_to_bitboard(*, file_path, num_positions=10, num_omit=5, seed_num=0, fp):
    '''
    Acts as the driver function that reads the games and call all other functions to
    eventually write the:
        - bitboard representation 
        - label
    into a pickle file
    '''
    # Set seed and print current time
    set_seed(seed_num)
    print("Start Time =", datetime.now().strftime("%H:%M:%S"))     
    
    # Initialize Variables
    dataframe = {
    'bitboard':[],
    'label':[]
    }
    pgn = open(file_path)
    game = chess.pgn.read_game(pgn)
    counter = 0
    file_num = 1
    
    # Go through all games in given pgn file
    while game is not None:
        result = int(outcome(game))
        # If results is not a draw, then we record the games
        # result = 0 means white won
        # result = 1 means black won
        # result = 2 means draw
        if result != 2:
            position_list, move_list = pgn_to_fen(game)
            position_list = sample_position(position_list, move_list, num_positions, num_omit)
            for pos in position_list:
                dataframe['bitboard'].append(fen_to_bitboard(pos))
                dataframe['label'].append(result)
                counter += 1
                
                # Every record_num moves, we write to a pickle file (using file_num to track
                # the updated pickle file) and clear our dataframe
                record_num = 10000
                if counter%record_num == 0:
                    write_to_pickle(fp+'_'+str(file_num)+'.pkl', dataframe)
                    print('Total Moves Recorded: {}, {} moves recorded in {}, Time: {}'.format(counter, record_num, fp+'_'+str(file_num)+'.pkl', datetime.now().strftime('%H:%M:%S')))
                    file_num += 1
                    dataframe = {
                    'bitboard':[],
                    'label':[]
                    }  
                            
        # Read the next game (chess.pgn.read_game() is a generator)
        game = chess.pgn.read_game(pgn)
    
    # Write the remaining moves to a pkl file
    write_to_pickle(fp+'_'+str(file_num)+'.pkl', dataframe)
    print('Total Moves Recorded: {}, Remaining {} moves recorded in {}, Time: {}'.format(counter, len(dataframe['bitboard']), fp+'_'+str(file_num)+'.pkl', datetime.now().strftime('%H:%M:%S')))
    print()
    
def write_to_pickle(fp, dataframe):
    with open(fp, 'wb') as f:
        pickle.dump(dataframe, f)

In [3]:
games_to_bitboard(file_path='../data/pgn/CCRL-4040.[1244261].pgn', fp='../data/bitboard/bitboard')

Start Time = 00:18:58
Total Moves Recorded: 10000, 10000 moves recorded in ../data/bitboard/bitboard_1.pkl, Time: 00:23:51
Total Moves Recorded: 20000, 10000 moves recorded in ../data/bitboard/bitboard_2.pkl, Time: 00:28:45
Total Moves Recorded: 30000, 10000 moves recorded in ../data/bitboard/bitboard_3.pkl, Time: 00:33:14
Total Moves Recorded: 40000, 10000 moves recorded in ../data/bitboard/bitboard_4.pkl, Time: 00:37:25
Total Moves Recorded: 50000, 10000 moves recorded in ../data/bitboard/bitboard_5.pkl, Time: 00:41:20
Total Moves Recorded: 60000, 10000 moves recorded in ../data/bitboard/bitboard_6.pkl, Time: 00:45:18
Total Moves Recorded: 70000, 10000 moves recorded in ../data/bitboard/bitboard_7.pkl, Time: 00:49:17
Total Moves Recorded: 80000, 10000 moves recorded in ../data/bitboard/bitboard_8.pkl, Time: 00:53:04
Total Moves Recorded: 90000, 10000 moves recorded in ../data/bitboard/bitboard_9.pkl, Time: 00:56:49
Total Moves Recorded: 100000, 10000 moves recorded in ../data/bitboar

Total Moves Recorded: 810000, 10000 moves recorded in ../data/bitboard/bitboard_81.pkl, Time: 05:28:10
Total Moves Recorded: 820000, 10000 moves recorded in ../data/bitboard/bitboard_82.pkl, Time: 05:32:09
Total Moves Recorded: 830000, 10000 moves recorded in ../data/bitboard/bitboard_83.pkl, Time: 05:35:52
Total Moves Recorded: 840000, 10000 moves recorded in ../data/bitboard/bitboard_84.pkl, Time: 05:39:40
Total Moves Recorded: 850000, 10000 moves recorded in ../data/bitboard/bitboard_85.pkl, Time: 05:43:43
Total Moves Recorded: 860000, 10000 moves recorded in ../data/bitboard/bitboard_86.pkl, Time: 05:47:31
Total Moves Recorded: 870000, 10000 moves recorded in ../data/bitboard/bitboard_87.pkl, Time: 05:51:22
Total Moves Recorded: 880000, 10000 moves recorded in ../data/bitboard/bitboard_88.pkl, Time: 05:54:57
Total Moves Recorded: 890000, 10000 moves recorded in ../data/bitboard/bitboard_89.pkl, Time: 05:58:48
Total Moves Recorded: 900000, 10000 moves recorded in ../data/bitboard/bi

Total Moves Recorded: 1600000, 10000 moves recorded in ../data/bitboard/bitboard_160.pkl, Time: 10:20:52
Total Moves Recorded: 1610000, 10000 moves recorded in ../data/bitboard/bitboard_161.pkl, Time: 10:24:20
Total Moves Recorded: 1620000, 10000 moves recorded in ../data/bitboard/bitboard_162.pkl, Time: 10:27:39
Total Moves Recorded: 1630000, 10000 moves recorded in ../data/bitboard/bitboard_163.pkl, Time: 10:31:09
Total Moves Recorded: 1640000, 10000 moves recorded in ../data/bitboard/bitboard_164.pkl, Time: 10:34:36
Total Moves Recorded: 1650000, 10000 moves recorded in ../data/bitboard/bitboard_165.pkl, Time: 10:38:06
Total Moves Recorded: 1660000, 10000 moves recorded in ../data/bitboard/bitboard_166.pkl, Time: 10:41:59
Total Moves Recorded: 1670000, 10000 moves recorded in ../data/bitboard/bitboard_167.pkl, Time: 10:45:51
Total Moves Recorded: 1680000, 10000 moves recorded in ../data/bitboard/bitboard_168.pkl, Time: 10:49:39
Total Moves Recorded: 1690000, 10000 moves recorded in 

Total Moves Recorded: 2390000, 10000 moves recorded in ../data/bitboard/bitboard_239.pkl, Time: 15:10:05
Total Moves Recorded: 2400000, 10000 moves recorded in ../data/bitboard/bitboard_240.pkl, Time: 15:14:13
Total Moves Recorded: 2410000, 10000 moves recorded in ../data/bitboard/bitboard_241.pkl, Time: 15:17:59
Total Moves Recorded: 2420000, 10000 moves recorded in ../data/bitboard/bitboard_242.pkl, Time: 15:21:56
Total Moves Recorded: 2430000, 10000 moves recorded in ../data/bitboard/bitboard_243.pkl, Time: 15:26:58
Total Moves Recorded: 2440000, 10000 moves recorded in ../data/bitboard/bitboard_244.pkl, Time: 15:30:50
Total Moves Recorded: 2450000, 10000 moves recorded in ../data/bitboard/bitboard_245.pkl, Time: 15:34:32
Total Moves Recorded: 2460000, 10000 moves recorded in ../data/bitboard/bitboard_246.pkl, Time: 15:38:42
Total Moves Recorded: 2470000, 10000 moves recorded in ../data/bitboard/bitboard_247.pkl, Time: 15:42:30
Total Moves Recorded: 2480000, 10000 moves recorded in 

Total Moves Recorded: 3180000, 10000 moves recorded in ../data/bitboard/bitboard_318.pkl, Time: 20:21:32
Total Moves Recorded: 3190000, 10000 moves recorded in ../data/bitboard/bitboard_319.pkl, Time: 20:25:15
Total Moves Recorded: 3200000, 10000 moves recorded in ../data/bitboard/bitboard_320.pkl, Time: 20:29:44
Total Moves Recorded: 3210000, 10000 moves recorded in ../data/bitboard/bitboard_321.pkl, Time: 20:33:32
Total Moves Recorded: 3220000, 10000 moves recorded in ../data/bitboard/bitboard_322.pkl, Time: 20:37:08
Total Moves Recorded: 3230000, 10000 moves recorded in ../data/bitboard/bitboard_323.pkl, Time: 20:41:20
Total Moves Recorded: 3240000, 10000 moves recorded in ../data/bitboard/bitboard_324.pkl, Time: 20:44:58
Total Moves Recorded: 3250000, 10000 moves recorded in ../data/bitboard/bitboard_325.pkl, Time: 20:48:45
Total Moves Recorded: 3260000, 10000 moves recorded in ../data/bitboard/bitboard_326.pkl, Time: 20:52:43
Total Moves Recorded: 3270000, 10000 moves recorded in 

Total Moves Recorded: 3970000, 10000 moves recorded in ../data/bitboard/bitboard_397.pkl, Time: 01:48:41
Total Moves Recorded: 3980000, 10000 moves recorded in ../data/bitboard/bitboard_398.pkl, Time: 01:52:27
Total Moves Recorded: 3990000, 10000 moves recorded in ../data/bitboard/bitboard_399.pkl, Time: 01:55:53
Total Moves Recorded: 4000000, 10000 moves recorded in ../data/bitboard/bitboard_400.pkl, Time: 01:59:38
Total Moves Recorded: 4010000, 10000 moves recorded in ../data/bitboard/bitboard_401.pkl, Time: 02:03:24
Total Moves Recorded: 4020000, 10000 moves recorded in ../data/bitboard/bitboard_402.pkl, Time: 02:07:11
Total Moves Recorded: 4030000, 10000 moves recorded in ../data/bitboard/bitboard_403.pkl, Time: 02:11:12
Total Moves Recorded: 4040000, 10000 moves recorded in ../data/bitboard/bitboard_404.pkl, Time: 02:15:00
Total Moves Recorded: 4050000, 10000 moves recorded in ../data/bitboard/bitboard_405.pkl, Time: 02:18:36
Total Moves Recorded: 4060000, 10000 moves recorded in 

Total Moves Recorded: 4760000, 10000 moves recorded in ../data/bitboard/bitboard_476.pkl, Time: 14:32:05
Total Moves Recorded: 4770000, 10000 moves recorded in ../data/bitboard/bitboard_477.pkl, Time: 14:36:03
Total Moves Recorded: 4780000, 10000 moves recorded in ../data/bitboard/bitboard_478.pkl, Time: 14:39:22
Total Moves Recorded: 4790000, 10000 moves recorded in ../data/bitboard/bitboard_479.pkl, Time: 14:43:17
Total Moves Recorded: 4800000, 10000 moves recorded in ../data/bitboard/bitboard_480.pkl, Time: 14:47:00
Total Moves Recorded: 4810000, 10000 moves recorded in ../data/bitboard/bitboard_481.pkl, Time: 14:50:43
Total Moves Recorded: 4820000, 10000 moves recorded in ../data/bitboard/bitboard_482.pkl, Time: 14:54:44
Total Moves Recorded: 4830000, 10000 moves recorded in ../data/bitboard/bitboard_483.pkl, Time: 14:58:32
Total Moves Recorded: 4840000, 10000 moves recorded in ../data/bitboard/bitboard_484.pkl, Time: 15:02:15
Total Moves Recorded: 4850000, 10000 moves recorded in 

Total Moves Recorded: 5550000, 10000 moves recorded in ../data/bitboard/bitboard_555.pkl, Time: 19:30:21
Total Moves Recorded: 5560000, 10000 moves recorded in ../data/bitboard/bitboard_556.pkl, Time: 19:34:13
Total Moves Recorded: 5570000, 10000 moves recorded in ../data/bitboard/bitboard_557.pkl, Time: 19:37:49
Total Moves Recorded: 5580000, 10000 moves recorded in ../data/bitboard/bitboard_558.pkl, Time: 19:41:54
Total Moves Recorded: 5590000, 10000 moves recorded in ../data/bitboard/bitboard_559.pkl, Time: 19:45:38
Total Moves Recorded: 5600000, 10000 moves recorded in ../data/bitboard/bitboard_560.pkl, Time: 19:49:43
Total Moves Recorded: 5610000, 10000 moves recorded in ../data/bitboard/bitboard_561.pkl, Time: 19:53:47
Total Moves Recorded: 5620000, 10000 moves recorded in ../data/bitboard/bitboard_562.pkl, Time: 19:57:29
Total Moves Recorded: 5630000, 10000 moves recorded in ../data/bitboard/bitboard_563.pkl, Time: 20:01:24
Total Moves Recorded: 5640000, 10000 moves recorded in 

Total Moves Recorded: 6340000, 10000 moves recorded in ../data/bitboard/bitboard_634.pkl, Time: 00:24:51
Total Moves Recorded: 6350000, 10000 moves recorded in ../data/bitboard/bitboard_635.pkl, Time: 00:28:54
Total Moves Recorded: 6360000, 10000 moves recorded in ../data/bitboard/bitboard_636.pkl, Time: 00:32:38
Total Moves Recorded: 6370000, 10000 moves recorded in ../data/bitboard/bitboard_637.pkl, Time: 00:36:36
Total Moves Recorded: 6380000, 10000 moves recorded in ../data/bitboard/bitboard_638.pkl, Time: 00:40:16
Total Moves Recorded: 6390000, 10000 moves recorded in ../data/bitboard/bitboard_639.pkl, Time: 00:44:33
Total Moves Recorded: 6400000, 10000 moves recorded in ../data/bitboard/bitboard_640.pkl, Time: 00:48:30
Total Moves Recorded: 6410000, 10000 moves recorded in ../data/bitboard/bitboard_641.pkl, Time: 00:52:16
Total Moves Recorded: 6420000, 10000 moves recorded in ../data/bitboard/bitboard_642.pkl, Time: 00:56:15
Total Moves Recorded: 6430000, 10000 moves recorded in 

Total Moves Recorded: 7130000, 10000 moves recorded in ../data/bitboard/bitboard_713.pkl, Time: 05:09:15
Total Moves Recorded: 7140000, 10000 moves recorded in ../data/bitboard/bitboard_714.pkl, Time: 05:12:36
Total Moves Recorded: 7150000, 10000 moves recorded in ../data/bitboard/bitboard_715.pkl, Time: 05:16:00
Total Moves Recorded: 7160000, 10000 moves recorded in ../data/bitboard/bitboard_716.pkl, Time: 05:19:22
Total Moves Recorded: 7170000, 10000 moves recorded in ../data/bitboard/bitboard_717.pkl, Time: 05:22:51
Total Moves Recorded: 7180000, 10000 moves recorded in ../data/bitboard/bitboard_718.pkl, Time: 05:26:23
Total Moves Recorded: 7190000, 10000 moves recorded in ../data/bitboard/bitboard_719.pkl, Time: 05:29:51
Total Moves Recorded: 7200000, 10000 moves recorded in ../data/bitboard/bitboard_720.pkl, Time: 05:33:05
Total Moves Recorded: 7210000, 10000 moves recorded in ../data/bitboard/bitboard_721.pkl, Time: 05:36:40
Total Moves Recorded: 7220000, 10000 moves recorded in 

In [53]:
# dataframe['identifier'].append(str(game.headers))